드라이브 마운트 확인

소스코드 참조

 https://www.kaggle.com/hamishdickson/cnn-for-sentence-classification-by-yoon-kim

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
path = "/content/drive/My Drive/2020 1학기/자연어처리/friends"
print(os.listdir(path))

['en_data.csv', 'en_sample.csv', 'friends_train.json', 'friends_dev.json', 'friends_test.json', '.ipynb_checkpoints', 'model.h5', 'model_by_label.h5', 'en_bert_results.csv', 'en_bert_results_round.csv', 'en_results.csv', 'en_results_round.csv', 'en_results_6.csv', 'en_results_round_6.csv', 'en_results_9.csv', 'en_results_round_9.csv', 'en_results_round_3.csv', 'en_results_3.csv', 'lstm_2_model.h5', 'lstm_2_model_25.h5', 'nlp_en_cnn_3.csv']


임포트

In [ ]:
import numpy as np
import pandas as pd
import json

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from keras.models import Model, load_model
from keras.layers import *
from keras.utils.np_utils import to_categorical
import re

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


전처리

In [ ]:
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
nltk.download('stopwords')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

# text 를 전처리 해서 쓰려고 만들었는데 막상 적용해보니 성능이 떨어져서 비활성화함
def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

# 대략적인 전처리
def preprocess_dataframe(df):
  df = df.dropna() # none 제거해줌

  for i in df.index:
    i_utterance = df.at[i, 'i_utterance']
    i_dialog = df.at[i, 'i_dialog']
    utterance = df.at[i, 'utterance']

    if i_utterance == 0: # 새로운 dialog 시작되면
      context = "" #초기화
    else:
      context = context + " " + utterance
    
    # 한 dialog 의 텍스트를 다 붙여놓은 형태. 이전 대화의 context도 포함하기 위함
    df.at[i, 'context'] = context
    
  # 길이 
  df['l'] = df['context'].apply(lambda x: len(str(x).split(' ')))
  print("mean length of sentence: " + str(df.l.mean()))
  print("max length of sentence: " + str(df.l.max()))
  print("std dev length of sentence: " + str(df.l.std()))
  return df

# json 파일을 읽어서, kaggle sample csv를 읽은 것처럼 포맷을 정리해준다
def read_json_make_dataframe(name='train'):  
  file = open('%s/friends_%s.json' % (path, name), encoding='latin1')
  d = json.load(file)
  data = []
  columns = ['i_dialog', 'i_utterance', 'speaker', 'utterance', 'annotation', 'emotion']
  i_dialog = -1
  for dialog in d:
    i_dialog += 1 # 0부터 시작
    i_utterance = -1
    for row in dialog:
      i_utterance += 1 # 0부터 시작
      data.append([
                   i_dialog, 
                   i_utterance, 
                   row['speaker'], 
                   row['utterance'], 
                   str(row['annotation']), 
                   row['emotion']
                   ])
      
        

  df = pd.DataFrame(data, columns=columns)
  df = preprocess_dataframe(df)
  return df

# kaggle data set 을 읽기 위한 함수
def read_csv_make_dataframe():
  df = pd.read_csv('%s/en_data.csv' % path, encoding='latin1')
  df = preprocess_dataframe(df)
  df = df[['i_dialog', 'i_utterance', 'speaker', 'utterance', 'context']]
  return df


train = read_json_make_dataframe('train')
print(train)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
mean length of sentence: 63.43404980588959
max length of sentence: 303
std dev length of sentence: 49.02061864789282
       i_dialog  ...    l
0             0  ...    1
1             0  ...    7
2             0  ...   13
3             0  ...   22
4             0  ...   27
...         ...  ...  ...
10556       719  ...  129
10557       719  ...  139
10558       719  ...  146
10559       719  ...  147
10560       719  ...  160

[10561 rows x 8 columns]


annotaion 변환

In [ ]:
# annotation 값을 emotion 값으로 변환함
def annotation_to_emotion(annotation):
    # 반올림 기반 변환법

    # 5 곱해서 반올림 친 벡터를 만듦
    rounded = [round(a * 5) for a in annotation]    

    # non-neutral check
    # 최대치가 같은게 두개 이상 있으면 neutral
    max_value = max(rounded)
    if len([x for x in rounded if x == max_value]) >= 2:
      return "non-neutral"
    
    # 맥스 밸류 인덱스에 해당하는 이모션 리턴
    return ['neutral','joy', 'sadness', 'fear', 'anger', 'surprise', 'disgust'][rounded.index(max_value)]

def summary_y_hats(y_hats):
  # y_hats의 분포를 보여주는 함수
  count_dict = {}
  
  for i in range(0, len(y_hats)):
    y_hat = y_hats[i]
    prediected_emotion = annotation_to_emotion(y_hat) # 예측한 이모션 값
    count_dict[prediected_emotion] = count_dict.get(prediected_emotion, 0) + 1

  print("summary of y:", count_dict)
  return
  

러닝 프로세스

In [ ]:
# 예측
def predict(data):
  from sklearn.metrics import f1_score, accuracy_score
  count = len(data)
  X = generate_X(data)
  y = generate_Y(data) # 정답
  y_hats = model.predict(X) # 예측치(annotation 벡터)

  print("")
  print("summary of y")
  summary_y_hats(y)

  print("")
  print("summary of y_hats")
  summary_y_hats(y_hats)

  y_true = [x for x in data['emotion'].values]
  y_pred = [annotation_to_emotion(x) for x in y_hats]

  acc = accuracy_score(y_true, y_pred)
  print("accuray: ", acc)
  print("f1_score: ",f1_score(y_true, y_pred, average='macro'))
  return acc

def run(index=""):
  data = read_csv_make_dataframe()

  count = len(data)
  X = generate_X(data)
  y_hats = model.predict(X) # 예측치(annotation 벡터)

  results = []
  columns = ['Id', 'Predicted']
  for i in range(0, count):
    y_hat = y_hats[i]
    prediected_emotion = annotation_to_emotion(y_hat) # 예측한 이모션 값

    results.append((i, prediected_emotion))

  pd.DataFrame(results, columns=columns).to_csv('%s/nlp_en_cnn_%s.csv' % (path, index), index=False)
  return

def learn():
  # 테스트 데이터 읽음
  test = read_json_make_dataframe('test')
  acc = predict(test)
  accuracies = []
  for i in range(0, 3):
    epochs = 3
    batch_size = 32

    history = model.fit(X_train, 
                        y_train, 
                        epochs=epochs, 
                        batch_size=batch_size,
                        shuffle=True,
    )
    acc = predict(test)
    accuracies.append(acc)
    learned = epochs * (i+1)
    run(learned)

  print("seq of accuracies:", accuracies)


읽은 인풋으로 러닝 데이터를 생성한다.

In [ ]:
# these sentences aren't that long so we may as well use the whole string
sequence_length = 70 # 이 길이를 넘은 문장은 걍 짤림
max_features = 20000 #len(embeddings_index) # this is the number of words we care about
tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', lower=False)

# X 데이터를 만들어줌
def generate_X(df):
  # tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', lower=False)
  tokenizer.fit_on_texts(df['utterance'].values)

  # this takes our sentences and replaces each word with an integer
  X = tokenizer.texts_to_sequences(df['utterance'].values)

  # we then pad the sequences so they're all the same length (sequence_length)
  X = pad_sequences(X, sequence_length)
  return X

# Y 데이터를 만들어줌
def generate_Y(df):
  annotations = df['annotation'].apply(lambda x: pd.Series([float(c)/5 for c in x], dtype='float32'))
  # 어노테이션 값을 벡터화함.
  # 2120000 ->벡터화-> [2, 1, 2, 0, 0, 0, 0] ->나누기 5-> [0.4, 0.2, 0.4, 0, 0, 0, 0]
  return annotations.values

X_train = generate_X(train)
y_train = generate_Y(train)

모델이 있다면 로딩하고, 없으면 생성해봄

In [ ]:
embedding_dim = 300 # Kim uses 300 here
num_filters = 100

def create_model():
  inputs = Input(shape=(sequence_length,), dtype='int32')

  # use a random embedding for the text
  embedding_layer = Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=sequence_length)(inputs)
  reshape = Reshape((sequence_length, embedding_dim, 1))(embedding_layer)
  from tensorflow.python.keras import regularizers
  # Note the relu activation which Kim specifically mentions
  # He also uses an l2 constraint of 3
  # Also, note that the convolution window acts on the whole 300 dimensions - that's important
  conv_0 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)
  conv_1 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)
  conv_2 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)

  # perform max pooling on each of the convoluations
  maxpool_0 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0)
  maxpool_1 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1)
  maxpool_2 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2)
  
  # concat and flatten
  concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2,])
  flatten = Flatten()(concatenated_tensor)

  # do dropout and predict
  dropout = Dropout(0.5)(flatten)
  output = Dense(units=7, activation='softmax')(dropout)

  model = Model(inputs=inputs, outputs=output)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  print(model.summary())
  return model

model = create_model()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 70)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 70, 300)      6000000     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 70, 300, 1)   0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 68, 1, 100)   90100       reshape_1[0][0]                  
____________________________________________________________________________________________

실행


In [ ]:
learn()

mean length of sentence: 60.91280752532562
max length of sentence: 286
std dev length of sentence: 47.66618446797404

summary of y
summary of y: {'surprise': 327, 'neutral': 1369, 'joy': 320, 'non-neutral': 321, 'anger': 190, 'sadness': 105, 'fear': 47, 'disgust': 85}

summary of y_hats
summary of y: {'non-neutral': 2764}
accuray:  0.19573082489146165
f1_score:  0.040922844175491684


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
10561/10561 [==============================] - 4s 355us/step - loss: 1.8528 - accuracy: 0.5504
Epoch 2/3
10561/10561 [==============================] - 3s 328us/step - loss: 1.6228 - accuracy: 0.5514
Epoch 3/3
10561/10561 [==============================] - 3s 328us/step - loss: 1.6323 - accuracy: 0.5514

summary of y
summary of y: {'surprise': 327, 'neutral': 1369, 'joy': 320, 'non-neutral': 321, 'anger': 190, 'sadness': 105, 'fear': 47, 'disgust': 85}

summary of y_hats
summary of y: {'neutral': 2764}
accuray:  0.46562952243125905
f1_score:  0.07942483337447544
mean length of sentence: 58.89381067961165
max length of sentence: 215
std dev length of sentence: 45.748533592551894
Epoch 1/3
10561/10561 [==============================] - 3s 330us/step - loss: 1.6383 - accuracy: 0.5514
Epoch 2/3
10561/10561 [==============================] - 4s 333us/step - loss: 1.6421 - accuracy: 0.5514
Epoch 3/3
10561/10561 [==============================] - 3s 331us/step - loss: 1.6415 - accur